In [ ]:
import os
import json
import pandas as pd

In [220]:
import os
import json
import pandas as pd

folderPathPorData = '..\\..\\Bases\\PorData\\'

# Função para ler o arquivo JSON e retornar seu conteúdo como uma lista de dicionários
def read_json_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"Error: O arquivo {file_path} não foi encontrado.")
    except json.JSONDecodeError:
        print(f"Error: Falha ao decodificar JSON no arquivo {file_path}.")
    return []

# Função para ler arquivos CSV a partir de uma pasta
def read_csv_files_from_folders(folder_path):
    dataframes = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.csv'):
                file_path = os.path.join(root, file)
                print(f'Processando arquivo: {file_path}')
                try:
                    df = pd.read_csv(file_path, sep=',', encoding='utf-8', low_memory=False)
                    dataframes.append({
                        'df_folder': os.path.basename(root),
                        'df_name': os.path.splitext(file)[0],
                        'df': df
                    })
                except Exception as e:
                    print(f"Erro ao ler {file}: {e}")
    return dataframes

# Função para tratar o DataFrame de cultura e cinemas
def trata_df_cultura(df_cultura_cinemas):
    df_cultura_cinemas = df_cultura_cinemas[df_cultura_cinemas['03. Âmbito Geográfico'].notnull()]
    last_year = df_cultura_cinemas['01. Ano'].max()
    df_cultura_cinemas = df_cultura_cinemas[df_cultura_cinemas['01. Ano'] == last_year]

    df_cultura_cinemas = df_cultura_cinemas[['01. Ano', '02. Nome Região (Portugal)', '03. Âmbito Geográfico', '09. Valor']]
    df_cultura_cinemas = df_cultura_cinemas.rename(columns={
        '01. Ano': 'Ano',
        '02. Nome Região (Portugal)': 'Região',
        '03. Âmbito Geográfico': 'Âmbito Geográfico',
        '09. Valor': 'Valor'
    })
    df_cultura_cinemas['Ano'] = df_cultura_cinemas['Ano'].astype(int)
    df_cultura_cinemas['Valor'] = df_cultura_cinemas['Valor'].astype(int)
    return df_cultura_cinemas.reset_index(drop=True)

# Função para encontrar um DataFrame pelo nome
def find_df_by_name(dataframes, df_name):
    for df_info in dataframes:
        if df_info['df_name'] == df_name:
            return df_info['df']
    raise ValueError(f"DataFrame com o nome '{df_name}' não encontrado.")

# Função para listar cidades com cinemas
def lista_cidades_cinemas(df_cultura_cinemas):
    df_cultura_cinemas = df_cultura_cinemas.groupby(['Região', 'Âmbito Geográfico']).agg({'Valor': 'sum'}).reset_index()
    df_cultura_cinemas = df_cultura_cinemas.sort_values(by='Valor', ascending=False).reset_index(drop=True)
    print("Cidades com mais cinemas:")
    for _, row in df_cultura_cinemas.iterrows():
        print(f"{row['Região']}: {row['Valor']} cinemas")

# Função para tratar o DataFrame de população residente
def trata_df_populacao_residente(df_populaccao_residente):
    df_populaccao_residente = df_populaccao_residente[df_populaccao_residente['04. Âmbito Geográfico'].notnull()]
    last_year = df_populaccao_residente['01. Ano'].max()
    df_populaccao_residente = df_populaccao_residente[df_populaccao_residente['01. Ano'] == last_year]

    df_sexo_pivot = df_populaccao_residente.pivot_table(
        index=['01. Ano', '03. Nome Região (Portugal)'],
        columns='05. Filtro 1',
        values='10. Valor',
        aggfunc='sum',
        fill_value=0
    )

    df_faixa_pivot = df_populaccao_residente.pivot_table(
        index=['01. Ano', '03. Nome Região (Portugal)'],
        columns='06. Filtro 2',
        values='10. Valor',
        aggfunc='sum',
        fill_value=0
    )

    df_polucao_final = pd.merge(df_sexo_pivot, df_faixa_pivot, on=['01. Ano', '03. Nome Região (Portugal)'], how='left')
    df_polucao_final.reset_index(inplace=True)
    df_polucao_final = df_polucao_final.rename(columns={
        '01. Ano': 'Ano',
        '03. Nome Região (Portugal)': 'Região',
        'Homens': 'Homens',
        'Mulheres': 'Mulheres',
        '0 - 4 anos': '0 - 4 anos',
        '5 - 9 anos': '5 - 9 anos',
        '10 - 14 anos': '10 - 14 anos',
        '15 - 19 anos': '15 - 19 anos',
        '20 - 24 anos': '20 - 24 anos',
        '25 - 29 anos': '25 - 29 anos',
        '30 - 34 anos': '30 - 34 anos',
        '35 - 39 anos': '35 - 39 anos',
        '40 - 44 anos': '40 - 44 anos',
        '45 - 49 anos': '45 - 49 anos',
        '50 - 54 anos': '50 - 54 anos',
        '55 - 59 anos': '55 - 59 anos',
        '60 - 64 anos': '60 - 64 anos',
        '65 - 69 anos': '65 - 69 anos',
        '70 - 74 anos': '70 - 74 anos',
        '75 - 79 anos': '75 - 79 anos',
        '80 - 84 anos': '80 - 84 anos',
        '85 ou mais anos': '85 ou mais anos',
        'Total_y': 'Total População'
    })

    df_polucao_final = df_polucao_final.astype({
        'Ano': 'int',
        'Região': 'str',
        'Homens': 'int',
        'Mulheres': 'int',
        '0 - 4 anos': 'int',
        '5 - 9 anos': 'int',
        '10 - 14 anos': 'int',
        '15 - 19 anos': 'int',
        '20 - 24 anos': 'int',
        '25 - 29 anos': 'int',
        '30 - 34 anos': 'int',
        '35 - 39 anos': 'int',
        '40 - 44 anos': 'int',
        '45 - 49 anos': 'int',
        '50 - 54 anos': 'int',
        '55 - 59 anos': 'int',
        '60 - 64 anos': 'int',
        '65 - 69 anos': 'int',
        '70 - 74 anos': 'int',
        '75 - 79 anos': 'int',
        '80 - 84 anos': 'int',
        '85 ou mais anos': 'int',
        'Total População': 'int'
    })

    # Calcular faixas etárias e proporções
    df_polucao_final['Total Crianças'] = df_polucao_final['0 - 4 anos'] + df_polucao_final['5 - 9 anos']
    df_polucao_final['Total Adolescentes'] = df_polucao_final['10 - 14 anos'] + df_polucao_final['15 - 19 anos']
    df_polucao_final['Total Jovens Adultos'] = df_polucao_final['20 - 24 anos']
    df_polucao_final['Total Adultos'] = df_polucao_final[['25 - 29 anos', '30 - 34 anos', '35 - 39 anos', '40 - 44 anos', 
                                                       '45 - 49 anos', '50 - 54 anos', '55 - 59 anos']].sum(axis=1)
    df_polucao_final['Total Idosos'] = df_polucao_final[['60 - 64 anos', '65 - 69 anos', '70 - 74 anos', '75 - 79 anos', 
                                                        '80 - 84 anos', '85 ou mais anos']].sum(axis=1)

    # Proporções de cada faixa etária
    df_polucao_final['Proporção Crianças'] = (df_polucao_final['Total Crianças'] / df_polucao_final['Total População']) * 100
    df_polucao_final['Proporção Adolescentes'] = (df_polucao_final['Total Adolescentes'] / df_polucao_final['Total População']) * 100
    df_polucao_final['Proporção Jovens Adultos'] = (df_polucao_final['Total Jovens Adultos'] / df_polucao_final['Total População']) * 100
    df_polucao_final['Proporção Adultos'] = (df_polucao_final['Total Adultos'] / df_polucao_final['Total População']) * 100
    df_polucao_final['Proporção Idosos'] = (df_polucao_final['Total Idosos'] / df_polucao_final['Total População']) * 100

    df_polucao_final = df_polucao_final.drop('Total_x', axis=1)

    return df_polucao_final.reset_index(drop=True)

# Função para listar cidades por grupo etário
def cidades_por_grupo(df, categoria='Todos', n=5):
    categorias = {
        'Crianças': 'Proporção Crianças',
        'Adolescentes': 'Proporção Adolescentes',
        'Jovens Adultos': 'Proporção Jovens Adultos',
        'Adultos': 'Proporção Adultos',
        'Idosos': 'Proporção Idosos',
        'Populosas': 'Total População'
    }
    
    if categoria not in categorias and categoria != 'Todos':
        print("Categoria inválida!")
        return
    
    if categoria == 'Todos':
        categorias_list = categorias.keys()
    else:
        categorias_list = [categoria]

    for cat in categorias_list:
        cidade_ordenada = df[['Ano', 'Região', categorias[cat]]].sort_values(by=categorias[cat], ascending=False).head(n)
        print(f"\nCidades com mais {cat} (%) - Top {n}:")
        for _, row in cidade_ordenada.iterrows():
            proporcao = row[categorias[cat]]
            print(f"{row['Região']}: {int(proporcao)}%")

def trata_df_populacao_densidade(df_densidade_populacional):

    df_densidade_populacional = df_densidade_populacional.dropna(axis=1, how='all')

    df_densidade_populacional = df_densidade_populacional[df_densidade_populacional['03. Nome Região (Portugal)'].notnull()]

    last_year = df_densidade_populacional['01. Ano'].max()
    df_densidade_populacional = df_densidade_populacional[df_densidade_populacional['01. Ano'] == last_year]

    df_densidade_populacional = df_densidade_populacional[df_densidade_populacional['03. Nome Região (Portugal)'] != 'Portugal']

    df_densidade_populacional = df_densidade_populacional[['01. Ano', '03. Nome Região (Portugal)', '04. Âmbito Geográfico', '10. Valor']]

    df_densidade_populacional.rename(columns={
        '01. Ano': 'Ano',
        '03. Nome Região (Portugal)': 'Região',
        '04. Âmbito Geográfico': 'Âmbito Geográfico',
        '10. Valor': 'Valor'
    }, inplace=True)

    df_densidade_populacional = df_densidade_populacional.astype({
        'Ano': 'int',
        'Região': 'str',
        'Âmbito Geográfico': 'str',
        'Valor': 'float'
    })

    return df_densidade_populacional.reset_index(drop=True)

In [ ]:
file_path = 'pordataFiles.json'
arquivos = read_json_file(file_path)

In [219]:
dataframes = read_csv_files_from_folders(folderPathPorData)

Processando arquivo: ..\..\Bases\PorData\cultura\623-recintos-de-cinema.csv
Processando arquivo: ..\..\Bases\PorData\cultura\631-sessoes-de-espetaculos-ao-vivo.csv
Processando arquivo: ..\..\Bases\PorData\economia\294-depositos-de-clientes-nos-bancos-caixas-economicas-e.csv
Processando arquivo: ..\..\Bases\PorData\economia\295-estabelecimentos-de-bancos-caixas-economicas-e-caixas.csv
Processando arquivo: ..\..\Bases\PorData\educacao\375-estabelecimentos-de-ensino-superior-por-subsistema.csv
Processando arquivo: ..\..\Bases\PorData\educacao\379-estabelecimentos-de-ensino-nao-superior-por-0.csv
Processando arquivo: ..\..\Bases\PorData\emprego\439-desemprego-registado-nos-centros-de-emprego-por-grup.csv
Processando arquivo: ..\..\Bases\PorData\emprego\445-populacao-empregada-por-conta-de-outrem-por-nivel-de.csv
Processando arquivo: ..\..\Bases\PorData\emprego\447-populacao-empregada-por-sexo-nivel-de-escolaridade-e.csv
Processando arquivo: ..\..\Bases\PorData\emprego\845-Pessoal-ao-servic

In [ ]:
df_cultura_cinemas = find_df_by_name(dataframes, '623-recintos-de-cinema')
df_cultura_cinemas = trata_df_cultura(df_cultura_cinemas)
lista_cidades_cinemas(df_cultura_cinemas)

In [ ]:
df_cultura_espetaculos = find_df_by_name(dataframes, '631-sessoes-de-espetaculos-ao-vivo')
df_cultura_espetaculos = trata_df_cultura(df_cultura_espetaculos)
lista_cidades_cinemas(df_cultura_espetaculos)

In [ ]:
df_populaccao_residente = find_df_by_name(dataframes, '1-populacao-residente-por-sexo-e-grupo-etario')
df_populaccao_final = trata_df_populacao_residente(df_populaccao_residente)
cidades_por_grupo(df_populaccao_final, categoria='Todos', n = 10) # Todos Crianças Adolescentes Jovens Adultos Adultos Idosos Populosas

In [ ]:
df_densidade_populacional = find_df_by_name(dataframes, '4-densidade-populacional')
df_densidade_populacional = trata_df_populacao_densidade(df_densidade_populacional)

In [213]:
df_densidade_populacional

,Ano,Região,Âmbito Geográfico,Valor
6790,2023,Abrantes,Município,47.3
6791,2023,Águeda,Município,140.8
6792,2023,Aguiar da Beira,Município,25.6
6793,2023,Alandroal,Município,9.2
6794,2023,Albergaria-a-Velha,Município,162.3
...,...,...,...,...
7093,2023,Vimioso,Município,8.7
7094,2023,Vinhais,Município,10.7
7095,2023,Viseu,Município,201.1
7096,2023,Vizela,Município,993.1
